In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden: Video Object Tracking with Bytetrack
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 CPU notebook is recommended)
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This Jupyter notebook provides a step-by-step walkthrough of deploying a video object tracking model on Vertex AI Endpoint resource with the open-source [ByteTrack](https://github.com/ifzhang/ByteTrack) object tracking algorithm.

### Objective
* Set up a Vertex AI Endpoint resource with:
    * TensorFlow Vision [notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb) or
    * Google Proprietary [notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb)
<br></br>
* Test integrated tracking model
    * Upload models to model registry
    * Deploy uploaded models
    * Run batch predictions
    * Verify and visualize tracking results
<br></br>
* Cleanup resources

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
! pip install --upgrade pip
! pip install fastapi==0.96.0
! pip install google-cloud-aiplatform==1.25.0
! pip install google-cloud-storage==2.9.0
! pip install tensorflow==2.11.0
! pip install uvicorn==0.22.0

### Colab Only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk) and [gsutil](https://cloud.google.com/storage/docs/gsutil_install).

## Before you begin

### Set up your Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project). Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

1. [Enable Artifact Registry](https://cloud.google.com/artifact-registry/docs/enable-service) and [create a repository](https://cloud.google.com/artifact-registry/docs/repositories/create-repos) for storing docker images.

1. [Create a GCS bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create?&_ga=2.233472348.-356102079.1688744268#iam-service-accounts-create-console) with Vertex AI User and Storage Object Admin roles for deploying fine tuned model to Vertex AI endpoint. [See how to grant Cloud Storage permissions to your service account](https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples).

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To update your model artifacts without re-building the container, you must upload your model
artifacts and any custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Setup remaining variables

In [ ]:
# Cloud project setup.

# The folder in the GCS bucket with input videos.
# Fill it without the 'gs://' prefix.
INPUT_GCS_FOLDER = ""  # @param {type:"string"}

# The video filename for the videos to be process.
# Fill it without the 'gs://' prefix.
VIDEO_FILE_NAME = ""  # @param {type:"string"}

# The video filename extension like .mp4. Please include period.
# Fill it without the 'gs://' prefix.
VIDEO_FILE_EXTENSION = ""  # @param {type:"string"}

# The folder in the GCS bucket where to store output videos and text
# annotations. Fill it without the 'gs://' prefix.
OUTPUT_GCS_FOLDER = ""  # @param {type:"string"}

# The Vertex IOD endpoint for object detection.
# It is like projects/<project_number>/locations/<location>/endpoints/<endpoint_id>"
DETECTION_ENDPOINT = ""  # @param {type:"string"}

# The label map for the Vertex IOD endpoint.
# It is the path to a .yaml in GCS.
# It is like gs://{BUCKET_NAME}/{FOLDER_NAME}/label_map.yaml
ENDPOINT_LABEL_MAP = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

# The pre-built docker images
SERVE_DOCKER_URI = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/vot-serve:latest"
)

# Prediction constants
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"

# The serving port.
SERVE_PORT = 7080

# The serving route.
SERVE_ROUTE = "/predictions/vot_serving"

# The service account you created in step-6 above.
# It is like "<account_name>@<project>.iam.gserviceaccount.com"
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### Initialize Vertex AI SDK

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
from google.cloud import aiplatform

# Init common setup.
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define utility functions


In [ ]:
def get_job_name_with_datetime(prefix: str):
    """
    Generate a job name string with the current date and time appended.

    Args:
        prefix: The prefix string to use for the job name.

    Returns:
        str: The job name string in the format "{prefix}_{YYYYMMDD_HHMMSS}".
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    project_id=None,
    detection_endpoint=None,
    label_map=None,
    output_bucket=None,
    model_type="CUSTOM",
    save_video_results=1,
):
    """
    Deploy a model to a real-time prediction endpoint.

    Args:
        detection_endpoint: The endpoint URL for object detection.
        label_map: Mapping of class IDs to class names.
        output_bucket: GCS bucket to save results.
        save_video_results: Whether to save video results.

    Returns:
        The created endpoint and deployed model objects.
    """
    task = "tracking"
    endpoint = aiplatform.Endpoint.create(display_name=f"{task}-endpoint")
    serving_env = {
        "MODEL_ID": "Bytetrack-Multi-Object-Tracking",
        "MODEL_TYPE": model_type,
        "PROJECT_ID": project_id,
        "DETECTION_ENDPOINT": detection_endpoint,
        "LABEL_MAP": label_map,
        "OUTPUT_BUCKET": output_bucket,
        "SAVE_VIDEO_RESULTS": save_video_results,
    }
    model = aiplatform.Model.upload(
        display_name=task,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[SERVE_PORT],
        serving_container_predict_route=SERVE_ROUTE,
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=PREDICTION_MACHINE_TYPE,
        accelerator_type=PREDICTION_ACCELERATOR_TYPE,
        accelerator_count=1,
        service_account=SERVICE_ACCOUNT,
    )
    return endpoint, model

### Video Object Tracking with Local Endpoint
This section shows how to deploy the chained IOD Vertex AI endpoint with tracking locally and obtain predictions saved in a text file. Users also have the option to save the annotated video to their GCS bucket.

* If you have not done so already, please set up a Vertex AI Endpoint resource with:
    * TensorFlow Vision [notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb) or
    * Google Proprietary [notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb)

In [ ]:
# The Vertex IOD endpoint for object detection.
# It is like projects/<project_number>/locations/<location>/endpoints/<endpoint_id>"
DETECTION_ENDPOINT = ""  # @param {type:"string"}

This is the local URL for making requests to the tracking model serving container running on localhost. It points to the /predictions route on port SERVE_PORT that will handle model inference requests.

In [ ]:
LOCAL_SERVE_URL = f"http://localhost:{SERVE_PORT}/{SERVE_ROUTE}"

### Run container with custom model IOD endpoint.

Run the serving container in a separate shell. The following environment variables are required:

* `DETECTION_ENDPOINT`: Vertex AI IOD detection endpoint address.
* `LABEL_MAP`: GCS uri of label map yaml file used to trained IOD.
* `OUTPUT_BUCKET`: GCS path for storing experiment outputs.

In [ ]:
!nvidia-docker run -t --rm \
-p {SERVE_PORT}:{SERVE_PORT} \
-e DETECTION_ENDPOINT=f"{DETECTION_ENDPOINT}" \
-e LABEL_MAP=f"{ENDPOINT_LABEL_MAP}" \
-e OUTPUT_BUCKET=f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}" \
-e SAVE_VIDEO_RESULTS=1 \
-e CUDA_VISIBLE_DEVICES=0 \
{SERVE_DOCKER_URI}

### Run container with AutoML trained model IOD endpoint.

Run the serving container in a separate shell. The following environment variables are required:

* `PROJECT_ID`: Google Cloud project ID.
* `MODEL_TYPE`: CUSTOM or AUTOML.
* `DETECTION_ENDPOINT`: Vertex AI IOD detection endpoint address from AUTOML trained IOD model.
* `OUTPUT_BUCKET`: GCS path for storing experiment outputs.

In [ ]:
!nvidia-docker run -t --rm \
-p {SERVE_PORT}:{SERVE_PORT} \
-e PROJECT_ID={PROJECT_ID} \
-e MODEL_TYPE={MODEL_TYPE}  \
-e DETECTION_ENDPOINT=f"{DETECTION_ENDPOINT}" \
-e OUTPUT_BUCKET=f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}" \
-e SAVE_VIDEO_RESULTS=1 \
-e CUDA_VISIBLE_DEVICES=0 \
{SERVE_DOCKER_URI}

### Test endpoint locally and perform online prediction
This section shows how to make prediction requests to the endpoint to obtain track IDs and bounding box coordinates for detected and tracked objects saved to a text file and/or annotated video output.

In [ ]:
import json

payload = json.dumps(
    {
        "instances": [
            {
                "video_uri": f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
            },
        ]
    }
)
r = requests.post(
    LOCAL_SERVE_URL,
    data=payload,
    headers={"content-type": "application/json", "Accept-Charset": "UTF-8"},
)
preds = r.json()

print(preds)

### Deploy Vertex AI Endpoint and custom IOD model.

In [ ]:
endpoint, model = deploy_model(
    detection_endpoint="",  # @param {type:"string"}
    label_map="",  # @param {type:"string"}
    output_bucket="",  # @param {type:"string"}
    model_type="CUSTOM",  # @param {type:"string"}
    save_video_results=1,
)

### Deploy Vertex AI Endpoint and AutoML trained IOD model.

In [ ]:
endpoint, model = deploy_model(
    project_id="",  # @param {type:"string"}
    detection_endpoint="",  # @param {type:"string"}
    model_type="AUTOML",  # @param {type:"string"}
    output_bucket="",  # @param {type:"string"}
)

### Perform online prediction with remote endpoint.

In [ ]:
instances = [
    {
        "data": {
            "video_uri": f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
        }
    },
]
preds = endpoint.predict(instances=instances).predictions
print(preds)

## Batch Prediction

### Setup input file for batch prediction and upload to gs bucket
Provide batch prediction input in jsonl format.

In [ ]:
INPUT_FILE = "instances.jsonl"
VIDEO_PATH_1 = (
    f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
)
VIDEO_PATH_2 = (
    f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}2{VIDEO_FILE_EXTENSION}"
)

In [ ]:
%%writefile $INPUT_FILE
{"data": { "video_uri": VIDEO_PATH_1}}
{"data": { "video_uri": VIDEO_PATH_2}}

In [ ]:
!gsutil cp "instances.jsonl" f"gs://{GCS_BUCKET}"

In [ ]:
gcs_input_uri = f"gs://{GCS_BUCKET}/instances.jsonl"
dest_uri = f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}"
print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### Create batch prediction job id

In [ ]:
JOB_PREFIX = "<job name prefix>"  # @param {type:"string"}
job_name = get_job_name_with_datetime(JOB_PREFIX)
print(job_name)

### Perform batch prediction

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=job_name,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=dest_uri,
    sync=False,
    machine_type=PREDICTION_MACHINE_TYPE,
    service_account=SERVICE_ACCOUNT,
)

print(batch_predict_job)

In [ ]:
batch_predict_job.wait()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI